In [141]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *

In [142]:
rules = pd.read_csv(config.files['service_rules'])


In [155]:
test = rules.groupby(['service_id', 'matching'])[['text_match']].apply(lambda x: ','.join(x))



text_match    text_match    text_match    text_match    text_match    text_match    text_match    text_match    text_match    text_match    text_match
------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
(1, 'A')      t             e             x             t             _             m             a             t             c             h
(1, 'S')      t             e             x             t             _             m             a             t             c             h
(3, 'A')      t             e             x             t             _             m             a             t             c             h
(3, 'R')      t             e             x             t             _             m             a             t             c             h
(3, 'S')      t             e             x             t             _             m             a             t             c 